# Inverse Problems with Physics-Informed Neural Networks
## Parameter Estimation in Heat Conduction

**Learning Objectives:**
- Understand the difference between forward and inverse problems
- Learn how PINNs solve inverse problems elegantly
- Master parameter estimation with sparse, noisy data
- Implement thermal diffusivity estimation from temperature measurements

## Introduction: Forward vs Inverse Problems

### The Forward Problem
In our previous PINN examples, we solved **forward problems**:
- **Given:** Complete physics (equations + parameters)
- **Find:** Solution function $u(x,t)$
- **Example:** Given spring constant $k$ and damping $c$, find oscillator motion $u(t)$

### The Inverse Problem  
In many real-world scenarios, we face **inverse problems**:
- **Given:** Some measurements of the solution $u$
- **Find:** Unknown physical parameters in the governing equations
- **Example:** From temperature measurements, determine thermal conductivity

### Why Inverse Problems are Hard

**Traditional Approach:**
1. Guess parameter values
2. Solve forward problem (expensive numerical simulation)
3. Compare with measurements
4. Update guess and repeat

**Problems:**
- Computationally expensive (many forward solves)
- Sensitive to noise
- May not converge or find wrong parameters
- Requires good initial guesses

**PINN Revolution:** Solve forward and inverse problems simultaneously!

![Inverse Problem Concept](https://github.com/chishiki-ai/sciml/blob/main/docs/01-pinn/figs/inverse-heat-diffusivity.png?raw=1)

## The Heat Equation: Our Test Case

We'll demonstrate inverse problems using **1D steady-state heat conduction**:

$$-k \frac{d^2T}{dx^2} = f(x), \quad x \in [0,1]$$

with boundary conditions: $T(0) = T_0$, $T(1) = T_1$

**Physical meaning:**
- $T(x)$ = temperature distribution
- $k$ = **thermal diffusivity** (unknown parameter we want to find)
- $f(x)$ = heat source term (known)
- Boundary conditions specify temperatures at the ends

### Why This Problem?
1. **Physical relevance:** Material property identification is crucial in engineering
2. **Mathematical simplicity:** 1D steady-state allows clear visualization
3. **Exact solution available:** We can verify our parameter recovery
4. **Well-studied:** Benchmark for inverse methods

### The Challenge
**Scenario:** You're an engineer testing a new material. You can:
- Apply known heat sources $f(x)$
- Control boundary temperatures
- Measure temperature at a few points
- **Cannot directly measure thermal diffusivity $k$**

**Goal:** Determine $k$ from sparse, noisy temperature measurements

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

# Set random seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)

# Problem setup
L = 1.0          # Domain length
T0, T1 = 0.0, 0.0  # Boundary temperatures (homogeneous)
kappa_true = 0.5   # True thermal diffusivity (unknown to the PINN)

def heat_source(x):
    """Known heat source term f(x)"""
    return -15*x + 2

def exact_temperature(x, kappa):
    """Analytical solution for validation"""
    # For -kappa * d²T/dx² = f(x) with f(x) = -15x + 2
    # and T(0) = T(1) = 0
    # Solution: T(x) = (15*x³ - 2*x² - 13*x) / (6*kappa)
    return (15*x**3 - 2*x**2 - 13*x) / (6*kappa)

print(f"Problem Setup:")
print(f"  Domain: [0, {L}]")
print(f"  Boundary conditions: T(0) = {T0}, T(1) = {T1}")
print(f"  Heat source: f(x) = -15x + 2")
print(f"  True thermal diffusivity: κ = {kappa_true}")
print(f"  Goal: Estimate κ from sparse temperature measurements")

## Stage 1: Understanding the Forward Problem

Before tackling the inverse problem, let's understand the forward problem and generate our "experimental" data.

### Analytical Solution
For our specific case with $f(x) = -15x + 2$ and homogeneous boundary conditions, the exact solution is:

$$T(x) = \frac{15x^3 - 2x^2 - 13x}{6\kappa}$$

**Key insight:** Notice how $T(x)$ depends on $\kappa$. Different values of $\kappa$ give different temperature profiles!

In [ ]:
# Visualize how κ affects temperature profiles
x_plot = np.linspace(0, 1, 200)
kappa_values = [0.2, 0.5, 1.0, 2.0]

plt.figure(figsize=(12, 5))

# Plot 1: Temperature profiles for different κ
plt.subplot(1, 2, 1)
for kappa in kappa_values:
    T_exact = exact_temperature(x_plot, kappa)
    plt.plot(x_plot, T_exact, linewidth=2, label=f'κ = {kappa}')

plt.xlabel('Position x')
plt.ylabel('Temperature T(x)')
plt.title('Temperature Profiles for Different Thermal Diffusivities')
plt.legend()
plt.grid(True, alpha=0.3)
plt.axhline(0, color='black', linewidth=0.5)

# Plot 2: Heat source
plt.subplot(1, 2, 2)
f_plot = heat_source(x_plot)
plt.plot(x_plot, f_plot, 'r-', linewidth=2, label='f(x) = -15x + 2')
plt.xlabel('Position x')
plt.ylabel('Heat Source f(x)')
plt.title('Known Heat Source Term')
plt.legend()
plt.grid(True, alpha=0.3)
plt.axhline(0, color='black', linewidth=0.5)

plt.tight_layout()
plt.show()

print("Key Observations:")
print("• Lower κ → Higher temperature magnitudes")
print("• Higher κ → Lower temperature magnitudes")
print("• Heat source is negative for x > 2/15 ≈ 0.133")
print("• This creates the characteristic temperature profile shape")

### Creating "Experimental" Data

In a real experiment, we would measure temperature at a few locations. Let's simulate this with sparse, noisy measurements.

In [ ]:
# Generate sparse, noisy experimental data
n_data = 10  # Only 10 measurement points!
noise_level = 0.05  # 5% noise

# Measurement locations (avoid boundaries where T=0)
x_data = np.linspace(0.1, 0.9, n_data)
T_data_exact = exact_temperature(x_data, kappa_true)

# Add noise to simulate experimental uncertainty
noise = noise_level * np.std(T_data_exact) * np.random.normal(0, 1, n_data)
T_data_noisy = T_data_exact + noise

# Plot the "experimental" setup
plt.figure(figsize=(10, 6))
T_true_plot = exact_temperature(x_plot, kappa_true)
plt.plot(x_plot, T_true_plot, 'k-', linewidth=2, label=f'True solution (κ = {kappa_true})')
plt.scatter(x_data, T_data_noisy, color='red', s=100, zorder=5,
           label=f'"Experimental" data ({n_data} points, {noise_level*100}% noise)')
plt.scatter([0, 1], [T0, T1], color='blue', s=100, marker='s', zorder=5,
           label='Boundary conditions')

plt.xlabel('Position x')
plt.ylabel('Temperature T(x)')
plt.title('Inverse Problem Setup: Estimate κ from Sparse Temperature Measurements')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print(f"Experimental Data Summary:")
print(f"  Measurement points: {n_data}")
print(f"  Noise level: {noise_level*100}%")
print(f"  True κ: {kappa_true} (unknown to PINN)")
print(f"  Temperature range: [{T_data_noisy.min():.3f}, {T_data_noisy.max():.3f}]")
print(f"\nChallenge: Can PINN recover κ = {kappa_true} from this sparse, noisy data?")

## Stage 2: The PINN Approach to Inverse Problems

### The Key Insight

**Traditional inverse methods:**
- Iterate between parameter guessing and forward solving
- Computationally expensive
- Prone to local minima

**PINN approach:**
- **Treat unknown parameters as additional trainable variables**
- Optimize network weights AND physical parameters simultaneously
- Use physics as regularization

### The PINN Inverse Framework

For our heat equation problem, we have:

**Unknowns to learn:**
1. $\hat{T}_\theta(x)$ - neural network approximating temperature
2. $\hat{\kappa}$ - estimated thermal diffusivity (scalar parameter)

**Loss function components:**

$$\mathcal{L}_{\text{total}}(\theta, \hat{\kappa}) = w_1\mathcal{L}_{\text{data}} + w_2\mathcal{L}_{\text{PDE}} + w_3\mathcal{L}_{\text{BC}}$$

where:

**Data Loss:** $\mathcal{L}_{\text{data}} = \frac{1}{N_d}\sum_{i=1}^{N_d}|\hat{T}_\theta(x_i) - T_i|^2$

**Physics Loss:** $\mathcal{L}_{\text{PDE}} = \frac{1}{N_f}\sum_{j=1}^{N_f}\left|-\hat{\kappa}\frac{d^2\hat{T}_\theta}{dx^2}(x_j) - f(x_j)\right|^2$

**Boundary Loss:** $\mathcal{L}_{\text{BC}} = |\hat{T}_\theta(0) - T_0|^2 + |\hat{T}_\theta(1) - T_1|^2$

### Why This Works

1. **Simultaneous optimization:** Both $\theta$ and $\hat{\kappa}$ are updated together
2. **Physics regularization:** The PDE constraint guides parameter estimation
3. **Data efficiency:** Physics fills gaps between sparse measurements
4. **Robustness:** Physics constraints filter noise in data

## Stage 3: Implementation

### Neural Network Architecture

We use a simple feedforward network for the temperature field:

In [ ]:
class HeatPINN(nn.Module):
    """PINN for inverse heat equation with parameter estimation"""

    def __init__(self, hidden_size=32, n_layers=3, kappa_init=0.1):
        super().__init__()

        # Neural network for temperature field T(x)
        layers = []
        layers.append(nn.Linear(1, hidden_size))  # Input: position x

        for _ in range(n_layers):
            layers.append(nn.Tanh())  # Smooth activation for PDE
            layers.append(nn.Linear(hidden_size, hidden_size))

        layers.append(nn.Tanh())
        layers.append(nn.Linear(hidden_size, 1))  # Output: temperature T

        self.network = nn.Sequential(*layers)

        # Trainable parameter: thermal diffusivity κ
        # Use log parameterization to ensure κ > 0
        self.log_kappa = nn.Parameter(torch.tensor(np.log(kappa_init), dtype=torch.float32))

        print(f"PINN Architecture:")
        print(f"  Network: {1} → {hidden_size} → ... → {1} ({n_layers} hidden layers)")
        print(f"  Parameter: κ initialized to {kappa_init}")
        print(f"  Total parameters: {sum(p.numel() for p in self.parameters())}")

    def forward(self, x):
        """Forward pass: compute temperature T(x)"""
        return self.network(x)

    @property
    def kappa(self):
        """Return positive thermal diffusivity"""
        return torch.exp(self.log_kappa)

    def physics_residual(self, x):
        """Compute PDE residual: -κ * d²T/dx² - f(x)"""
        x = x.clone().detach().requires_grad_(True)

        # Forward pass
        T = self.forward(x)

        # First derivative: dT/dx
        dT_dx = torch.autograd.grad(T, x, grad_outputs=torch.ones_like(T),
                                   create_graph=True, retain_graph=True)[0]

        # Second derivative: d²T/dx²
        d2T_dx2 = torch.autograd.grad(dT_dx, x, grad_outputs=torch.ones_like(dT_dx),
                                     create_graph=True, retain_graph=True)[0]

        # Heat source term
        f = heat_source(x.detach().numpy()).reshape(-1, 1)
        f_tensor = torch.tensor(f, dtype=torch.float32)

        # PDE residual: -κ * d²T/dx² - f(x) = 0
        residual = -self.kappa * d2T_dx2 - f_tensor

        return residual

# Create the PINN model
kappa_init = 0.1  # Initial guess (far from true value of 0.5)
model = HeatPINN(kappa_init=kappa_init)

print(f"\nInitial parameter guess: κ = {model.kappa.item():.3f}")
print(f"True parameter value: κ = {kappa_true}")
print(f"Initial error: {abs(model.kappa.item() - kappa_true)/kappa_true*100:.1f}%")

### Training Function

The training process optimizes both the neural network weights $theta$ and the parameter $kappa$ simultaneously:

In [ ]:
def train_inverse_pinn(model, x_data, T_data, epochs=10000, lr=1e-3):
    """
    Train PINN for inverse parameter estimation

    Args:
        model: HeatPINN model
        x_data, T_data: Experimental measurements
        epochs: Number of training epochs
        lr: Learning rate
    """

    # Convert data to tensors
    x_data_tensor = torch.tensor(x_data.reshape(-1, 1), dtype=torch.float32)
    T_data_tensor = torch.tensor(T_data.reshape(-1, 1), dtype=torch.float32)

    # Collocation points for physics
    n_colloc = 100
    x_colloc = torch.linspace(0, 1, n_colloc).reshape(-1, 1)

    # Boundary points
    x_boundary = torch.tensor([[0.0], [1.0]], dtype=torch.float32)
    T_boundary = torch.tensor([[T0], [T1]], dtype=torch.float32)

    # Optimizer
    optimizer = optim.Adam(model.parameters(), lr=lr)

    # Loss weights
    w_data = 1.0
    w_pde = 1.0
    w_bc = 10.0  # Higher weight for boundary conditions

    # Storage for tracking
    loss_history = []
    kappa_history = []
    data_losses = []
    pde_losses = []
    bc_losses = []

    print(f"Training Inverse PINN:")
    print(f"  Data points: {len(x_data)}")
    print(f"  Collocation points: {n_colloc}")
    print(f"  Loss weights: data={w_data}, PDE={w_pde}, BC={w_bc}")
    print(f"  Starting κ = {model.kappa.item():.4f}")

    for epoch in tqdm(range(epochs), desc="Training"):
        optimizer.zero_grad()

        # Data loss: fit measurements
        T_pred_data = model(x_data_tensor)
        loss_data = torch.mean((T_pred_data - T_data_tensor)**2)

        # PDE loss: satisfy physics
        residual = model.physics_residual(x_colloc)
        loss_pde = torch.mean(residual**2)

        # Boundary loss: satisfy boundary conditions
        T_pred_bc = model(x_boundary)
        loss_bc = torch.mean((T_pred_bc - T_boundary)**2)

        # Total loss
        total_loss = w_data * loss_data + w_pde * loss_pde + w_bc * loss_bc

        # Backpropagation
        total_loss.backward()
        optimizer.step()

        # Store history
        loss_history.append(total_loss.item())
        kappa_history.append(model.kappa.item())
        data_losses.append(loss_data.item())
        pde_losses.append(loss_pde.item())
        bc_losses.append(loss_bc.item())

        if (epoch + 1) % 2000 == 0:
            print(f"Epoch {epoch+1:5d}: κ={model.kappa.item():.4f}, "
                  f"Total={total_loss.item():.6f}, "
                  f"Data={loss_data.item():.6f}, "
                  f"PDE={loss_pde.item():.6f}, "
                  f"BC={loss_bc.item():.6f}")

    return {
        'loss_history': loss_history,
        'kappa_history': kappa_history,
        'data_losses': data_losses,
        'pde_losses': pde_losses,
        'bc_losses': bc_losses
    }

## Stage 4: Training and Parameter Recovery

**The moment of truth!** Can the PINN simultaneously learn the temperature field AND recover the unknown thermal diffusivity?

In [ ]:
# Train the inverse PINN
training_results = train_inverse_pinn(model, x_data, T_data_noisy)

# Extract results
kappa_final = model.kappa.item()
kappa_error = abs(kappa_final - kappa_true) / kappa_true * 100

print(f"\n🎯 PARAMETER ESTIMATION RESULTS")
print(f"="*40)
print(f"True κ:        {kappa_true:.4f}")
print(f"Initial guess: {kappa_init:.4f}")
print(f"Final estimate: {kappa_final:.4f}")
print(f"Absolute error: {abs(kappa_final - kappa_true):.4f}")
print(f"Relative error: {kappa_error:.2f}%")

if kappa_error < 5.0:
    print(f"✅ Excellent parameter recovery!")
elif kappa_error < 10.0:
    print(f"✅ Good parameter recovery!")
else:
    print(f"⚠️  Parameter recovery needs improvement")

## Stage 5: Results Analysis

### Parameter Convergence History

Let's visualize how the parameter estimate converged during training:

In [ ]:
# Create comprehensive results visualization
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Plot 1: Parameter convergence
ax = axes[0, 0]
ax.plot(training_results['kappa_history'], 'b-', linewidth=2, label='Estimated κ')
ax.axhline(kappa_true, color='red', linestyle='--', linewidth=2, label=f'True κ = {kappa_true}')
ax.axhline(kappa_init, color='gray', linestyle=':', alpha=0.7, label=f'Initial guess = {kappa_init}')
ax.set_xlabel('Epoch')
ax.set_ylabel('Thermal Diffusivity κ')
ax.set_title('Parameter Convergence During Training')
ax.legend()
ax.grid(True, alpha=0.3)

# Plot 2: Loss components
ax = axes[0, 1]
ax.plot(training_results['loss_history'], 'r-', linewidth=2, label='Total loss', alpha=0.8)
ax.plot(training_results['data_losses'], 'b-', linewidth=1, label='Data loss', alpha=0.7)
ax.plot(training_results['pde_losses'], 'g-', linewidth=1, label='PDE loss', alpha=0.7)
ax.plot(training_results['bc_losses'], 'orange', linewidth=1, label='BC loss', alpha=0.7)
ax.set_xlabel('Epoch')
ax.set_ylabel('Loss')
ax.set_title('Training Loss Evolution')
ax.set_yscale('log')
ax.legend()
ax.grid(True, alpha=0.3)

# Plot 3: Temperature field comparison
ax = axes[1, 0]
with torch.no_grad():
    x_plot_tensor = torch.tensor(x_plot.reshape(-1, 1), dtype=torch.float32)
    T_pred_plot = model(x_plot_tensor).numpy().flatten()

T_true_plot = exact_temperature(x_plot, kappa_true)
T_pred_with_true_kappa = exact_temperature(x_plot, kappa_final)

ax.plot(x_plot, T_true_plot, 'k-', linewidth=3, label=f'True solution (κ={kappa_true})', alpha=0.8)
ax.plot(x_plot, T_pred_plot, 'b-', linewidth=2, label=f'PINN prediction (κ={kappa_final:.3f})', alpha=0.8)
ax.scatter(x_data, T_data_noisy, color='red', s=80, zorder=5, label='Training data')
ax.scatter([0, 1], [T0, T1], color='blue', s=80, marker='s', zorder=5, label='Boundary conditions')
ax.set_xlabel('Position x')
ax.set_ylabel('Temperature T(x)')
ax.set_title('Temperature Field: Prediction vs Truth')
ax.legend()
ax.grid(True, alpha=0.3)

# Plot 4: Parameter error over time
ax = axes[1, 1]
kappa_errors = [abs(k - kappa_true)/kappa_true * 100 for k in training_results['kappa_history']]
ax.plot(kappa_errors, 'purple', linewidth=2)
ax.set_xlabel('Epoch')
ax.set_ylabel('Relative Error in κ (%)')
ax.set_title('Parameter Estimation Error Over Time')
ax.set_yscale('log')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Compute temperature field accuracy
T_error = np.abs(T_pred_plot - T_true_plot)
T_rmse = np.sqrt(np.mean(T_error**2))
T_max_error = np.max(T_error)

print(f"\n🌡️  TEMPERATURE FIELD ACCURACY")
print(f"="*35)
print(f"RMSE: {T_rmse:.6f}")
print(f"Max error: {T_max_error:.6f}")
print(f"Relative RMSE: {T_rmse/np.std(T_true_plot)*100:.2f}%")

### Sensitivity Analysis: Effect of Data Sparsity

**Important question:** How does the number of measurement points affect parameter recovery?

Let's test with different amounts of data:

In [ ]:
# Sensitivity analysis: data sparsity vs parameter recovery
n_data_values = [3, 5, 10, 15, 20]
kappa_estimates = []
kappa_errors = []

print("🔬 SENSITIVITY ANALYSIS: Data Sparsity vs Parameter Recovery")
print("="*65)

for n_data_test in n_data_values:
    print(f"\nTesting with {n_data_test} data points...")

    # Generate test data
    x_test = np.linspace(0.1, 0.9, n_data_test)
    T_test_exact = exact_temperature(x_test, kappa_true)
    T_test_noisy = T_test_exact + noise_level * np.std(T_test_exact) * np.random.normal(0, 1, n_data_test)

    # Create and train model
    model_test = HeatPINN(kappa_init=0.1)

    # Shorter training for sensitivity analysis
    # The `with torch.no_grad():` block has been removed from around this call.
    results_test = train_inverse_pinn(model_test, x_test, T_test_noisy, epochs=5000, lr=1e-3)

    kappa_est = model_test.kappa.item()
    kappa_err = abs(kappa_est - kappa_true) / kappa_true * 100

    kappa_estimates.append(kappa_est)
    kappa_errors.append(kappa_err)

    print(f"  Estimated κ: {kappa_est:.4f} (error: {kappa_err:.2f}%)")

# Plot sensitivity analysis results
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Plot 1: Parameter estimates vs data points
ax1.plot(n_data_values, kappa_estimates, 'bo-', linewidth=2, markersize=8, label='PINN estimates')
ax1.axhline(kappa_true, color='red', linestyle='--', linewidth=2, label=f'True κ = {kappa_true}')
ax1.set_xlabel('Number of Data Points')
ax1.set_ylabel('Estimated κ')
ax1.set_title('Parameter Estimate vs Data Sparsity')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: Error vs data points
ax2.plot(n_data_values, kappa_errors, 'ro-', linewidth=2, markersize=8)
ax2.set_xlabel('Number of Data Points')
ax2.set_ylabel('Relative Error in κ (%)')
ax2.set_title('Parameter Error vs Data Sparsity')
ax2.set_yscale('log')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n📊 SENSITIVITY ANALYSIS RESULTS")
print(f"="*35)
for i, n in enumerate(n_data_values):
    print(f"{n:2d} points: κ = {kappa_estimates[i]:.4f}, error = {kappa_errors[i]:5.2f}%")

print(f"\n💡 Key Insights:")
print(f"• More data generally improves parameter recovery")
print(f"• Even with just 3 points, PINN achieves reasonable accuracy")
print(f"• Physics constraints help when data is extremely sparse")
print(f"• Diminishing returns beyond ~10-15 points for this problem")

## Summary: The Power of PINNs for Inverse Problems

### What We've Demonstrated

1. **Simultaneous Learning:** PINNs can learn both the solution field $T(x)$ and unknown parameters $\kappa$ in a single optimization process,
    
2. **Data Efficiency:** Excellent parameter recovery from just 10 noisy measurements across the entire domain

3. **Physics as Regularization:** The PDE constraint guides parameter estimation and filters noise

4. **Robustness:** Works well even with sparse, noisy data

### Why This is Revolutionary

**Traditional approach problems:**
- Requires many expensive forward solves
- Sensitive to initial guesses  
- Prone to local minima
- Struggles with noise

**PINN advantages:**
- Single optimization loop
- Physics provides strong regularization
- Handles noise naturally
- Works with minimal data

### Key Technical Insights

1. **Parameter parameterization:** Using `log(κ)` ensures positivity constraints
2. **Loss balancing:** Boundary conditions often need higher weights
3. **Collocation points:** Dense physics sampling compensates for sparse data
4. **Automatic differentiation:** Enables exact PDE residual computation

### Real-World Applications

**Material characterization:**
- Thermal conductivity from temperature measurements
- Elastic moduli from displacement data
- Permeability from pressure measurements

**Process monitoring:**
- Reaction rates from concentration data
- Heat transfer coefficients from thermal data
- Mass transfer coefficients from composition data

**Geophysics:**
- Subsurface properties from surface measurements
- Aquifer parameters from well data
- Seismic velocity from travel times

### The Broader Impact

PINNs transform inverse problems from:
- **Expensive iterative procedures** → **Single optimization**
- **Data-hungry methods** → **Physics-informed learning**
- **Noise-sensitive approaches** → **Robust estimation**
- **Domain-specific solvers** → **Universal framework**

**Next frontier:** Multi-parameter estimation, time-dependent problems, and coupled physics!

---

**🎯 Challenge:** Try modifying the code to estimate multiple parameters simultaneously (e.g., both $\kappa$ and the heat source amplitude)!